# Chapter 02 — Forces

- Newton's second law: acceleration = net_force / mass; integrate to update velocity/position.
- Common forces: gravity, drag, friction, springs (Hooke's law).
- Forces accumulate as vectors each frame; reset after integration.
- Force strength often scales with distance (inverse square for gravity).

Try: Experiment with drag coefficient or starting distance to see stable/unstable motion.

In [ ]:
import mathclass Vector:    def __init__(self, x=0.0, y=0.0):        self.x, self.y = float(x), float(y)    def __add__(self, other):        return Vector(self.x + other.x, self.y + other.y)    def __sub__(self, other):        return Vector(self.x - other.x, self.y - other.y)    def __mul__(self, k):        return Vector(self.x * k, self.y * k)    __rmul__ = __mul__    def mag(self):        return math.hypot(self.x, self.y)    def normalize(self):        m = self.mag()        if m == 0:            return Vector()        return self * (1.0 / m)    def limit(self, max_mag):        m = self.mag()        if m > max_mag:            return self.normalize() * max_mag        return self    def __repr__(self):        return f"Vector({self.x:.2f}, {self.y:.2f})"class Body:    def __init__(self, pos, mass=1.0):        self.pos = pos        self.vel = Vector(0, 0)        self.acc = Vector(0, 0)        self.mass = mass    def apply_force(self, force):        # F = m * a => a = F / m        self.acc += force * (1 / self.mass)    def update(self, dt=1.0):        self.vel += self.acc * dt        self.pos += self.vel * dt        self.acc = Vector(0, 0)def gravity(attractor, mover, G=1.0):    dir_vec = attractor.pos - mover.pos    dist = max(0.5, dir_vec.mag())    strength = G * attractor.mass * mover.mass / (dist * dist)    return dir_vec.normalize() * strengthattractor = Body(Vector(0, 0), mass=20)mover = Body(Vector(5, 0), mass=2)for step in range(5):    g = gravity(attractor, mover, G=0.8)    drag = mover.vel * -0.05    mover.apply_force(g)    mover.apply_force(drag)    mover.update(0.5)    print(f"step {step}: pos={mover.pos} vel={mover.vel}")